In [8]:
import requests
import traceback
import os
import boto3
import pandas as pd
from dotenv import load_dotenv
from github_python_fetch import fetch_function
from query_package import get_query_package
from datetime import datetime


# GitHub Access 
token = os.getenv('GITHUBTOKEN')

# AWS Access
access = os.getenv('ACCESS')
secret = os.getenv('SECRET')

# RDS Access
username = os.getenv('USER')
password = os.getenv('PASS')
server = os.getenv('SERVER')
db = os.getenv('DB')

bucket = os.getenv('BUCKET')    # AWS RDS bucket name
project_folder = os.getenv('PROJECTFOLDER')     # AWS S3 Project Folder Path
active_folder = os.getenv('ACTIVEFOLDER')    # AWS S3 Project Active Folder Path
archive_folder = os.getenv('ARCHIVEFOLDER')

# Access RDS Functions via GitHub
rds_connector_url = 'https://raw.githubusercontent.com/CharlesRoss-MBakerIntl/TB-RDS-Connect/main/rds_connector.py' # Set url to python file of github
rds_connector = fetch_function(rds_connector_url, token) # Pull function from github using requests
exec(rds_connector) # Execute the file


# Access RDS Functions via GitHub
s3_manager_url = 'https://raw.githubusercontent.com/CharlesRoss-MBakerIntl/TB-S3-Manager/main/s3_bucket_util.py' # Set url to python file of github
s3_manager = fetch_function(s3_manager_url, token) # Pull function from github using requests
exec(s3_manager) # Execute the file

# Create a Boto3 client for S3
s3 = boto3.client(
    's3',
    aws_access_key_id=access,
    aws_secret_access_key=secret
)


# Pull Query Package from File
query_package = get_query_package()


# Store RDS Functions Locally
conn, cursor = rds_connection(username, password, db, server) # Connect to RDS Database
rds = RDS(conn, cursor, query_package) # Create Instance of RDS Table
df = rds.df

In [13]:
add_archive_folder(s3 = s3, 
                   bucket = bucket, 
                   project_folder = project_folder, 
                   archive_folder = archive_folder, 
                   limit = 30,
                   versions = rds.cleaning_versions)


In [2]:
s3.put_object(Bucket = bucket, Key = project_folder + active_folder + "Active-FDEM-Hotem-Summary.csv", Body = pd.DataFrame().to_csv())

{'ResponseMetadata': {'RequestId': 'ZJ011A888TW6YAC9',
  'HostId': 'sFQT3m8x4MO+7RsbDp4UQcIT2k3o+s04519qXIMoWnvN8mXFPhnTKCVyHzOfzEkNa/wlueZXRDM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'sFQT3m8x4MO+7RsbDp4UQcIT2k3o+s04519qXIMoWnvN8mXFPhnTKCVyHzOfzEkNa/wlueZXRDM=',
   'x-amz-request-id': 'ZJ011A888TW6YAC9',
   'date': 'Thu, 31 Oct 2024 22:30:44 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"0ae9bcd0c0b0aa5aab99d84beca26ce8"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'ETag': '"0ae9bcd0c0b0aa5aab99d84beca26ce8"',
 'ServerSideEncryption': 'AES256'}